In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('./dataset/forestfires.csv')
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,3,6,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,10,3,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,10,7,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,3,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,3,1,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [3]:
#X needs to be tranposed because we want the data points to be the columns and rows to be the feature
X = np.transpose(df.iloc[:, :-1].values) #data matrix X, 12 x 517 (12 variables/features with 517 number of data points)
label = df.iloc[:, -1].values #the last column of the matrix. 517-vector.
# X = StandardScaler().fit_transform(X) #standardize the data
X.shape

(12, 517)

In [4]:
mean_vector = np.mean(X, axis=1)
mean_matrix = np.tile(mean_vector, (X.shape[1],1))
X = X - mean_matrix.T

In [5]:
SX = (np.dot(X, X.T))/(X.shape[1])#covariance matrix of X, 12 x 12
# SX = np.cov(X)
n = SX.shape[0] # dimension of the original space
q = 5 #dimension of the lower-dimensional subspace
SX.shape

(12, 12)

In [6]:
#calculate the eigenvalues and the corresponding eigenvectors for SX.
#no need to worry for complex eigenvalues as per Spectral Theorem, a real symmetric matrix only has real eigenvalues.
#the eigenvalues are not necessarily ordered.
eigenvalue, eigenvector = np.linalg.eig(SX)

In [7]:
eigval_eigvec = list(zip(*sorted(zip(eigenvalue,eigenvector), reverse=True))) #order eigenvalues in a descending manner.

In [8]:
principal_components = np.asarray(list(eigval_eigvec[1][:])).T #this needs to be transposed to put each eigenvector in rows instead of columns

In [9]:
P = principal_components[:q] # q x 12 matrix with q leading principal components in each row.

In [10]:
Y = np.dot(P, X) #transform X using P to get the reconstructed data points
Y.shape # the reconstructed data matrix shape

(5, 517)

In [11]:
variance_loss = sum(eigval_eigvec[0][q:]) #calculate the total loss
total_variance = sum(eigval_eigvec[0][:]) #total variance
print("The total loss of PCA is : ", (variance_loss/total_variance)*100, "%") #percentage of loss

The total loss of PCA is :  0.037985574963925774 %


In [12]:
from sklearn.linear_model import LinearRegression #to compare the linear regression prediction on the original data points and the reconstructed data points.
import datetime

In [13]:
start_time = datetime.datetime.now()
reg_original_data = LinearRegression().fit(X.T,label) #fit the original data with the label
print("The R2 coefficient is : ", reg_original_data.score(X.T, label)) #calculate the R2 coefficient on the training data itself
end_time = datetime.datetime.now()
print("Total elapsed time : ", end_time - start_time)

The R2 coefficient is :  0.025350671349257503
Total elapsed time :  0:00:00.002062


In [14]:
start_time = datetime.datetime.now()
reg_pca_data = LinearRegression().fit(Y.T, label) #fit the reconstructed data points with the label
print("The R2 coefficient is : ", reg_pca_data.score(Y.T, label)) # calculate the R2 coefficient with the training data itself 
end_time = datetime.datetime.now()
print("Total elapsed time : ", end_time - start_time)

The R2 coefficient is :  0.012739904147630043
Total elapsed time :  0:00:00.001400


In [15]:
'''
it can be seen that even though the reconstructed data points has a lower R2 coefficient, it saves some time
during the modelling of the linear regression. The time saved might be small for this example, but it makes a lot
of difference when it comes to higher dimensional dataset and more complex modelling techniques. Linear Regression
was just used for simplicity sake to show the working principles of PCA. That is, even though the dimension
from the original data points has been reduced drastically, the reconstructed data points can still provide
linear regression enough information to model the relationships between the dependent and independent variables.
'''
from sklearn.decomposition import PCA #using PCA from the sklearn 

In [16]:
pca = PCA(n_components=q)
pca.fit(X.T)
Y_2 = np.transpose(pca.transform(X.T))
Y_2.shape

(5, 517)

In [17]:
'''
the difference between the principal components are not that much. Sklearn's PCA uses normalizations on the 
data, hence provides a slightly different result than my implementation.
'''
prin_comp_diff = sum(sum(pca.components_) - sum(P)) #difference between the principal components or my implementation and Sklearn's implementation
print(prin_comp_diff)

0.597090255567526
